In [1]:
import pickle
parsed_parameter_save_path='C:/Users/saurav/Desktop/HCL assignment/parsed_parameters.pickle'
with open(parsed_parameter_save_path, 'rb') as f:
    temp=pickle.load(f)
temp

[{'query': 'total number of cases found in Afganistan?',
  'Place': {'no_match': ['afganistan'], 'states': [], 'countries': []},
  'Time Duration': {'begin': '01-01-2020', 'end': '31-12-2020'},
  'Case Type': 'confirm',
  'Function Type': 'sum',
  'Operation Type': 'cases'},
 {'query': 'total number of cases found in Colombia till july?',
  'Place': {'no_match': [], 'states': [], 'countries': ['colombia']},
  'Time Duration': {'begin': '01-07-2020', 'end': '31-07-2020'},
  'Case Type': 'confirm',
  'Function Type': 'sum',
  'Operation Type': 'cases'},
 {'query': 'total number of new cases found in France in april?',
  'Place': {'no_match': [], 'states': [], 'countries': ['france']},
  'Time Duration': {'begin': '01-04-2020', 'end': '31-04-2020'},
  'Case Type': 'confirm',
  'Function Type': 'sum',
  'Operation Type': 'cases'},
 {'query': 'total number of new cases found in Greece between april to september?',
  'Place': {'no_match': [], 'states': [], 'countries': ['greece']},
  'Time D

In [2]:
import sqlite3
temp={'query': 'total number of new cases recovered in india from march to december?',
  'Place': {'no_match': [], 'states': [], 'countries': ["'India'"]},
  'Time Duration': {'begin': "'2020-08-12'", 'end': "'2020-09-12'"},
  'Case Type': 'death',
  'Function Type': 'average',
  'Operation Type': 'cases'}

if temp['Case Type']=='confirm':
    types='confirmed'
elif temp['Case Type']=='recover':
    types='recovered'
elif temp['Case Type']=='death':
    types='deaths'
elif temp['Case Type']=='increase rate':
    types='Increase_rate'

In [3]:
con = sqlite3.connect(r"C:\Users\saurav\Desktop\H   C   L\database1\covid19.db")
if temp['Operation Type']=='cases':
    print(1)
    
    if len(temp['Place']['no_match'])==0 and len(temp['Place']['states'])==0 and len(temp['Place']['countries'])==0:
        
        
        if temp['Function Type']=='sum':
            cur = con.execute("SELECT "+types+" FROM "+"countries_aggregated "+"WHERE "+"Date="+temp['Time Duration']['begin']+";")
            a=cur.fetchone()
            cur = con.execute("SELECT "+types+" FROM "+"worldwide_aggregate "+"WHERE "+"Date="+temp['Time Duration']['end']+";")
            b=cur.fetchone()
            print(b[0]-a[0])
            
        elif temp['Function Type']=='maximum':
            cur = con.execute("SELECT "+types+" FROM "+"worldwide_aggregate "+"WHERE "+"Date "+"BETWEEN "+temp['Time Duration']['begin']+" AND "+temp['Time Duration']['end']+";")
            val=[]
            for row in cur:
                val.append(row[0]) 
            print(max(val[1:]))
            
        elif temp['Function Type']=='minimum':
            cur = con.execute("SELECT "+types+" FROM "+"worldwide_aggregate "+"WHERE "+"Date "+"BETWEEN "+temp['Time Duration']['begin']+" AND "+temp['Time Duration']['end']+";")
            val=[]
            for row in cur:
                val.append(row[0]) 
            print(min(val))
            
        elif temp['Function Type']=='average':
            cur = con.execute("SELECT "+types+" FROM "+"worldwide_aggregate "+"WHERE "+"Date "+"BETWEEN "+temp['Time Duration']['begin']+" AND "+temp['Time Duration']['end']+";")
            val=[]
            for row in cur:
                val.append(row[0]) 
            print(sum(val)/len(val))
            
            
    elif len(temp['Place']['no_match'])==0 and len(temp['Place']['states'])==0 and len(temp['Place']['countries'])!=0:
        
        
        if temp['Function Type']=='sum':
            cur = con.execute("SELECT "+types+" FROM "+"countries_aggregated "+"WHERE "+"Country="+temp['Place']['countries'][0]+" AND "+"Date="+temp['Time Duration']['begin']+";")
            a=cur.fetchone()
            cur = con.execute("SELECT "+types+" FROM "+"countries_aggregated "+"WHERE "+"Country="+temp['Place']['countries'][0]+" AND "+"Date="+temp['Time Duration']['end']+";")
            b=cur.fetchone()
            print(b[0]-a[0])
            
        elif temp['Function Type']=='maximum':
            cur = con.execute("SELECT "+types+" FROM "+"countries_aggregated "+"WHERE "+"Country="+temp['Place']['countries'][0]+" AND "+"Date "+"BETWEEN "+temp['Time Duration']['begin']+" AND "+temp['Time Duration']['end']+";")
            val=[]
            for row in cur:
                val.append(row[0]) 
            print(max(val))
            
        elif temp['Function Type']=='minimum':
            cur = con.execute("SELECT "+types+" FROM "+"countries_aggregated "+"WHERE "+"Country="+temp['Place']['countries'][0]+" AND "+"Date "+"BETWEEN "+temp['Time Duration']['begin']+" AND "+temp['Time Duration']['end']+";")
            val=[]
            for row in cur:
                val.append(row[0]) 
            print(min(val))
            
        elif temp['Function Type']=='average':
            cur = con.execute("SELECT "+types+" FROM "+"countries_aggregated "+"WHERE "+"Country="+temp['Place']['countries'][0]+" AND "+"Date "+"BETWEEN "+temp['Time Duration']['begin']+" AND "+temp['Time Duration']['end']+";")
            val=[]
            for row in cur:
                val.append(row[0]) 
            print(sum(val)/len(val))
    
    
    elif len(temp['Place']['no_match'])==0 and len(temp['Place']['states'])!=0:
        
        
        cur1 = con.execute("SELECT EXISTS(SELECT Province_State FROM us WHERE "+"Province_State="+temp['Place']['states'][0]+");")
        cur2 = con.execute("SELECT EXISTS(SELECT Province_State FROM timeseries WHERE "+"Province_State="+temp['Place']['states'][0]+");")
        if cur1.fetchone()[0]==1:
            if types=='Recovered':
                print('No data found')
            elif temp['Function Type']=='sum':
                cur = con.execute("SELECT "+types+" FROM "+"us "+"WHERE "+"Province_State="+temp['Place']['states'][0]+" AND "+"Date="+temp['Time Duration']['begin']+";")
                a=cur.fetchone()
                cur = con.execute("SELECT "+types+" FROM "+"us "+"WHERE "+"Province_State="+temp['Place']['states'][0]+" AND "+"Date="+temp['Time Duration']['end']+";")
                b=cur.fetchone()
                print(b[0]-a[0])
            
            elif temp['Function Type']=='maximum':
                cur = con.execute("SELECT "+types+" FROM "+"us "+"WHERE "+"Province_State="+temp['Place']['states'][0]+" AND "+"Date "+"BETWEEN "+temp['Time Duration']['begin']+" AND "+temp['Time Duration']['end']+";")
                val=[]
                for row in cur:
                    val.append(row[0]) 
                print(max(val))
            
            elif temp['Function Type']=='minimum':
                cur = con.execute("SELECT "+types+" FROM "+"us "+"WHERE "+"Province_State="+temp['Place']['states'][0]+" AND "+"Date "+"BETWEEN "+temp['Time Duration']['begin']+" AND "+temp['Time Duration']['end']+";")
                val=[]
                for row in cur:
                    val.append(row[0]) 
                print(min(val))
            
            elif temp['Function Type']=='average':
                cur = con.execute("SELECT "+types+" FROM "+"us "+"WHERE "+"Province_State="+temp['Place']['states'][0]+" AND "+"Date "+"BETWEEN "+temp['Time Duration']['begin']+" AND "+temp['Time Duration']['end']+";")
                val=[]
                for row in cur:
                    val.append(row[0]) 
                print(sum(val)/len(val))
            
        elif cur2.fetchone()[0]==1:
            if temp['Function Type']=='sum':
                cur = con.execute("SELECT "+types+" FROM "+"timeseries "+"WHERE "+"Province_State="+temp['Place']['states'][0]+" AND "+"Date="+temp['Time Duration']['begin']+";")
                a=cur.fetchone()
                cur = con.execute("SELECT "+types+" FROM "+"timeseries "+"WHERE "+"Province_State="+temp['Place']['states'][0]+" AND "+"Date="+temp['Time Duration']['end']+";")
                b=cur.fetchone()
                print(b[0]-a[0])
            
            elif temp['Function Type']=='maximum':
                cur = con.execute("SELECT "+types+" FROM "+"timeseries "+"WHERE "+"Province_State="+temp['Place']['states'][0]+" AND "+"Date "+"BETWEEN "+temp['Time Duration']['begin']+" AND "+temp['Time Duration']['end']+";")
                val=[]
                for row in cur:
                    val.append(row[0]) 
                print(max(val))
            
            elif temp['Function Type']=='minimum':
                cur = con.execute("SELECT "+types+" FROM "+"timeseries "+"WHERE "+"Province_State="+temp['Place']['states'][0]+" AND "+"Date "+"BETWEEN "+temp['Time Duration']['begin']+" AND "+temp['Time Duration']['end']+";")
                val=[]
                for row in cur:
                    val.append(row[0]) 
                print(min(val))
            
            elif temp['Function Type']=='average':
                cur = con.execute("SELECT "+types+" FROM "+"timeseries "+"WHERE "+"Province_State="+temp['Place']['states'][0]+" AND "+"Date "+"BETWEEN "+temp['Time Duration']['begin']+" AND "+temp['Time Duration']['end']+";")
                val=[]
                for row in cur:
                    val.append(row[0]) 
                print(sum(val)/len(val))
        
        else:
            print('No match')
    

elif temp['Operation Type']=='country':
    print('ye krna hai')
    
    
elif temp['Operation Type']=='state':
    print('ye bhi krna hai')
    
    
else:
    print('no match')

1
62252.21875
